### USCRN Data: High-Octane Scraping

In [1]:
import requests
import pandas as pd 
import yaml 
import re
import itertools
from datetime import datetime
from bs4 import BeautifulSoup

with open ("sources.yaml", "r") as yaml_file:
  sources = yaml.load(yaml_file, Loader=yaml.FullLoader)

##### 1.) Scrape Column Headers and Descriptions 

In [6]:
header_url = sources['USCRN']['headers']
header_response = requests.get(header_url)
header_soup = BeautifulSoup(header_response.content, "html.parser")

columns = str(header_soup).split("\n")[1].strip(" ").split(" ")
columns = list(map(lambda x: str.lower(x), columns)) # columns = [str.lower(c) for c in columns] -- faster?
columns.insert(0,'station_location')

descrip_text = str(header_soup).split("\n")[2] # raw text block containing column descriptions
descrip_text

"The station WBAN number. The UTC date of the observation. The UTC time of the observation. Time is the end of the observed hour, so the 0000 hour is actually the last hour of the previous day's observation (starting just after 11:00 PM through midnight). The Local Standard Time (LST) date of the observation. The Local Standard Time (LST) time of the observation. Time is the end of the observed hour (see UTC_TIME description). The version number of the station datalogger program that was in effect at the time of the observation. Note: This field should be treated as text (i.e. string). Station longitude, using WGS-84. Station latitude, using WGS-84. Average air temperature, in degrees C, during the last 5 minutes of the hour. See Note F. Average air temperature, in degrees C, for the entire hour. See Note F. Maximum air temperature, in degrees C, during the hour. See Note F. Minimum air temperature, in degrees C, during the hour. See Note F. Total amount of precipitation, in mm, record

The descriptions of the columns are quite the mess, as there is no standard separator used. We will have to work our way through it step by step: 

In [3]:
first_split = [re.sub(r'(\([^)]*)$', r"\1)", s) for s in descrip_text.split("). ")] # add ')' back after splitting text on ').' 
no_notes = [re.sub(r' See Note [A-Z]\.',"",s) for s in first_split] # drop any references to notes

The third entry in `no_notes` is ready. The last set of descriptions in `no_notes` can be split on `". "`, but the first two sets need special attention. We will pop the last set out and split it, then pop the third set out, and then address the first two sets. At that point we will recombine everything into one list while preserving the original order. 

In [4]:
last_set = no_notes.pop().strip().split(". ")
third_set = no_notes.pop() # Note: just a string

In [5]:
def flatten(ls:list): 
  return list(itertools.chain.from_iterable(ls)) 

no_notes = [re.sub(". Time is", " at", s) for s in no_notes]
first_second = flatten([s.split(". ") for s in no_notes])

# Finally:
descriptions = flatten([["Location name for USCRN station"], first_second, [third_set], last_set]) # Description added for "station_location" 

In [6]:
header_info = {
  'col_name': columns,
  'description': descriptions, 
  'units': ["X...(Various Lengths)","XXXXX", "YYYYMMDD", "HHmm", "YYYYMMDD", "HHmm", "XXXXXX", "Decimal_degrees", "Decimal_degrees", "Celsius", "Celsius", "Celsius", "Celsius", "mm", "W/m^2", "X", "W/m^2", "X", "W/m^2", "X", "X", "Celsius", "X", "Celsius", "X", "Celsius", "X", "%", "X", "m^3/m^3", "m^3/m^3", "m^3/m^3", "m^3/m^3", "m^3/m^3", "Celsius", "Celsius", "Celsius", "Celsius", "Celsius"]
}

header_df = pd.DataFrame(header_info)
# header_df.to_csv("data/column_descriptions.csv", index=True)

##### 2.) Scrape Core Data Files (>2 million rows)

In [2]:
base_url = sources["USCRN"]["index"]
base_response = requests.get(base_url)
base_soup = BeautifulSoup(base_response.content, "html.parser")

In [3]:
links = base_soup.find_all("a") # 'links' in this notebook will refer to <a> elements, not urls
years = [str(x).zfill(1) for x in range(2000,2024)]
year_links = [link for link in links if link['href'].rstrip('/') in years]

file_urls = []
for year_link in year_links: 
  year_url = base_url + year_link.get("href")
  response = requests.get(year_url) 
  soup = BeautifulSoup(response.content, 'html.parser')
  file_links = soup.find_all('a', href=re.compile(r'AK.*\.txt'))
  if file_links:
    new_file_urls = [year_url + link.getText() for link in file_links]
    file_urls.extend(new_file_urls)

In [4]:
rows = []
regex = r"([St.]*[A-Z][a-z]+_*[A-Za-z]*).*.txt" 
for url in file_urls:
  # Get location from url -- will add to BS results in next step
  file_name = re.search(regex, url).group(0)
  station_location = re.sub("(_formerly_Barrow.*|_[0-9].*)", "", file_name)
  # Get results 
  response = requests.get(url)
  soup = BeautifulSoup(response.content,'html.parser')
  soup_lines = [station_location + " " + line for line in str(soup).strip().split("\n")]
  #   soup_lines = [station_location + " " + line for line in str(soup).split("\n")] <-- Old Code 
  new_rows = [re.split('\s+', row) for row in soup_lines]
  rows.extend(new_rows)

In [7]:
df = pd.DataFrame(rows, columns=columns)

In [8]:
df.sample(10)

,station_location,wbanno,utc_date,utc_time,lst_date,lst_time,crx_vn,longitude,latitude,t_calc,...,soil_moisture_5,soil_moisture_10,soil_moisture_20,soil_moisture_50,soil_moisture_100,soil_temp_5,soil_temp_10,soil_temp_20,soil_temp_50,soil_temp_100
1784131,Fairbanks,26494,20210911,0000,20210910,1500,2.424,-147.51,64.97,16.1,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1911687,Toolik_Lake,96409,20210403,2000,20210403,1100,2.514,-149.40,68.65,-20.5,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
552166,Fairbanks,26494,20140824,0400,20140823,1900,2.424,-147.51,64.97,14.7,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
389735,Sand_Point,25630,20120110,0900,20120110,0000,2.404,-160.47,55.35,2.3,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1389116,Glennallen,56401,20190831,0100,20190830,1600,2.515,-145.50,63.03,17.4,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1655639,Ruby,96406,20200119,0400,20200118,1900,2.515,-154.13,64.50,-36.6,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
128618,Utqiagvik,27516,20070208,1600,20070208,0700,1.301,-156.61,71.32,-15.6,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1954350,Cordova,96405,20220215,1100,20220215,0200,2.514,-145.35,60.47,0.7,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1901005,Tok,96404,20210113,1800,20210113,0900,2.514,-141.21,62.74,-12.1,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1681659,Sitka,25379,20200105,0800,20200104,2300,2.424,-135.33,57.06,-2.1,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0


In [10]:
# Check for NaNs
df[df.isna().any(axis=1)]

,station_location,wbanno,utc_date,utc_time,lst_date,lst_time,crx_vn,longitude,latitude,t_calc,...,soil_moisture_5,soil_moisture_10,soil_moisture_20,soil_moisture_50,soil_moisture_100,soil_temp_5,soil_temp_10,soil_temp_20,soil_temp_50,soil_temp_100


Let's drop the soil temperature and moisture columns. These fields have missing data (-99.0 or -9999.0) for the vast majority of records.

In [11]:
soil_columns = df.filter(regex="soil")
soil_columns[soil_columns.isin([-99.0, -9999.0]).any(axis=1)]
soil_columns

,soil_moisture_5,soil_moisture_10,soil_moisture_20,soil_moisture_50,soil_moisture_100,soil_temp_5,soil_temp_10,soil_temp_20,soil_temp_50,soil_temp_100
0,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
3,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
4,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
...,...,...,...,...,...,...,...,...,...,...
2163427,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2163428,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2163429,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2163430,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0


In [9]:
df = df.filter(regex="^((?!soil).)*$")

Lastly, let's convert the date and time columns to `datetime` objects...

In [10]:
# pad time data with leading zeros
df['utc_time'] = df['utc_time'].astype(str).str.pad(4, fillchar='0')
df['lst_time'] = df['lst_time'].astype(str).str.pad(4, fillchar='0')

# create new columns with concatenated datetime values
df['utc_datetime'] = pd.to_datetime(df['utc_date'].astype(str) + df['utc_time'].fillna(''), format='%Y%m%d%H%M')
df['lst_datetime'] = pd.to_datetime(df['lst_date'].astype(str) + df['lst_time'].fillna(''), format='%Y%m%d%H%M')

# drop the original columns
df.drop(['utc_date', 'utc_time', 'lst_date', 'lst_time'], axis=1, inplace=True)

In [19]:
df.columns

Index(['station_location', 'wbanno', 'crx_vn', 'longitude', 'latitude',
       't_calc', 't_hr_avg', 't_max', 't_min', 'p_calc', 'solarad',
       'solarad_flag', 'solarad_max', 'solarad_max_flag', 'solarad_min',
       'solarad_min_flag', 'sur_temp_type', 'sur_temp', 'sur_temp_flag',
       'sur_temp_max', 'sur_temp_max_flag', 'sur_temp_min',
       'sur_temp_min_flag', 'rh_hr_avg', 'rh_hr_avg_flag', 'utc_datetime',
       'lst_datetime'],
      dtype='object')

...and add a `date_added` column: 

In [23]:
# first reorder columns
cols = ['station_location','wbanno', 'utc_datetime', 'lst_datetime', 'crx_vn', 'longitude', 'latitude'] + list(df.columns)[5:-2]
df = df[cols]
df['date_added_utc'] = datetime.now()

/home/alex/deb-projects/team-week3/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [26]:
df.sample(5)

,station_location,wbanno,utc_datetime,lst_datetime,crx_vn,longitude,latitude,t_calc,t_hr_avg,t_max,...,sur_temp_type,sur_temp,sur_temp_flag,sur_temp_max,sur_temp_max_flag,sur_temp_min,sur_temp_min_flag,rh_hr_avg,rh_hr_avg_flag,date_added_utc
1578349,Fairbanks,26494,2020-04-01 18:00:00,2020-04-01 09:00:00,2.424,-147.51,64.97,-6.6,-7.4,-6.6,...,C,-12.8,0,-11.4,0,-14.6,0,27,0,2023-02-17 12:00:54.760047
140488,Fairbanks,26494,2008-06-17 06:00:00,2008-06-16 21:00:00,1.302,-147.51,64.97,13.4,13.4,13.6,...,R,12.3,0,13.0,0,11.6,0,-9999,0,2023-02-17 12:00:54.760047
1630382,Metlakatla,25381,2020-03-04 19:00:00,2020-03-04 10:00:00,2.424,-131.59,55.05,2.4,2.4,3.1,...,C,2.2,0,2.7,0,1.4,0,80,0,2023-02-17 12:00:54.760047
1443658,Port_Alsworth,26562,2019-11-21 15:00:00,2019-11-21 06:00:00,2.424,-154.32,60.20,1.0,1.4,2.6,...,C,0.0,0,0.1,0,-0.1,0,90,0,2023-02-17 12:00:54.760047
902743,Port_Alsworth,26562,2016-11-27 19:00:00,2016-11-27 10:00:00,2.424,-154.32,60.20,-12.2,-11.8,-11.5,...,C,-12.7,0,-12.0,0,-13.7,0,78,0,2023-02-17 12:00:54.760047


In [15]:
df.to_csv("data/uscrn.csv", index=False)

Let's also make a table for our various station locations. This will be useful when searching for the four-day forecasts in the NWS notebook. 

In [1]:
locations = df[['station_location', 'wbanno', 'longitude', 'latitude']].drop_duplicates()
# locations.to_csv("data/locations.csv", index=False)

##### 3.) Upload Core Data to BigQuery 

In [79]:
%%bash
bq mk -d --location=us-east4 team-week3:alaska

Dataset 'team-week3:alaska' successfully created.


Core Data: 

In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Manually set schema to make columns required (NaN detection on upload) and avoid parsing certain columns as numeric (e.g. crx_vn, the flag columns)
# Setting these columns as strings will indicate that they are not made to have arithmetic calculations done on them
schema = [
  bigquery.SchemaField("station_location", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("wbanno", "STRING", mode="REQUIRED"),  
  bigquery.SchemaField("utc_datetime", "DATETIME", mode="REQUIRED"), 
  bigquery.SchemaField("lst_datetime", "DATETIME", mode="REQUIRED"), 
  bigquery.SchemaField("crx_vn", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("longitude", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("latitude", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("t_calc", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("t_hr_avg", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("t_max", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("t_min", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("p_calc", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("solarad", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("solarad_flag", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("solarad_max", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("solarad_max_flag", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("solarad_min", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("solarad_min_flag", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp_type", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp_flag", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp_max", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp_max_flag", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp_min", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp_min_flag", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("rh_hr_avg", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("rh_hr_avg_flag", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("date_added_utc", "DATETIME", mode="REQUIRED")
]

In [3]:
key_path = "/home/alex/.creds/alex-sa-tw3.json"
credentials = service_account.Credentials.from_service_account_file(
   key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id)

table_id = f"{credentials.project_id}.alaska.uscrn"

jc = bigquery.LoadJobConfig(
   source_format = bigquery.SourceFormat.CSV,
   autodetect=False,
   schema=schema,
   create_disposition="CREATE_IF_NEEDED",
   write_disposition="WRITE_TRUNCATE", 
   destination_table_description="Historical weather data from USCRN stations in Alaska"
)

job = client.load_table_from_dataframe(df, table_id, job_config=jc)

job.result()

ValueError: bq_schema contains fields not present in dataframe: {'longitude', 'crx_vn'}

##### 4.) DAG Task for Updating Dataset  

Pandas has a neat function for reading HTML tables to dataframes (`pd.read_html`). It's not ideal for "messier" tabular data, however, or for iterating through lots of HTML pages as we were doing earlier (iteratively creating and appending dataframes is very slow given the size of dataframe objects). But it's definitely useful for reading a single table object:   

In [10]:
now = datetime.now()
updates_url = sources['USCRN']['updates'] + f"{now.year}"

df = pd.read_html("https://www.ncei.noaa.gov/pub/data/uscrn/products/hourly02/updates/2023/", skiprows=[1,2])[0]
df.drop(["Size", "Description"], axis=1, inplace=True)
df.dropna(inplace=True)
cols = [re.sub(" ","_",str.lower(c)) for c in df.columns]
df.columns = cols
df['last_modified'] = pd.to_datetime(df['last_modified'])
df

,name,last_modified
0,CRN60H0203-202301010100.txt,2022-12-31 20:47:00
1,CRN60H0203-202301010200.txt,2022-12-31 21:47:00
2,CRN60H0203-202301010300.txt,2022-12-31 22:53:00
3,CRN60H0203-202301010400.txt,2022-12-31 23:54:00
4,CRN60H0203-202301010500.txt,2023-01-01 00:49:00
...,...,...
1138,CRN60H0203-202302171100.txt,2023-02-17 06:47:00
1139,CRN60H0203-202302171200.txt,2023-02-17 07:47:00
1140,CRN60H0203-202302171300.txt,2023-02-17 08:47:00
1141,CRN60H0203-202302171400.txt,2023-02-17 09:47:00


In [21]:
from collections import deque
from io import StringIO

with open("data/uscrn.csv", 'r') as fp:
    q = deque(fp, 1)  
last_added = pd.read_csv(StringIO(''.join(q)), header=None).iloc[0,-1]
last_added = datetime.strptime(last_added, "%Y-%m-%d %H:%M:%S.%f")

In [22]:
df[df['last_modified'] > last_added] # In actual use this won't be empty 

,name,last_modified


In [23]:
pd.read_csv(StringIO(''.join(q)), header=None)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,Yakutat,25382,2.424,-139.68,59.51,-0.5,-0.4,-0.3,-0.5,0.0,...,0,-1.9,0,-2.7,0,95,0,2023-02-17 13:00:00,2023-02-17 04:00:00,2023-02-17 15:59:40.186602


In [2]:
import pandas as pd 
df = pd.read_csv("data/uscrn.csv")